# Integration with a hardware synthesis tool for ML

In [ ]:
import os
import numpy as np
import tensorflow as tf 
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from qkeras import *
from qkeras import QActivation
from qkeras import QDense, QConv2DBatchnorm
import hls4ml

In [ ]:
# Path Vivado HLS 
# This should be configured based on the installation of Vitis HLS/Vivado HLS in your computer
os.environ['PATH'] = '/tools/Xilinx2019/Vivado/2019.2/bin:' + os.environ['PATH']
os.environ['PATH']


In [ ]:
# Load keras model 

#distilled_student_1D

model = load_model('../compressionAndTraining/models/distilled_student_2D.h5', custom_objects={'QConv2DBatchnorm':QConv2DBatchnorm,'quantized_bits':quantized_bits, 'QActivation': QActivation, 'QDense': QDense})
model.summary()

model.summary()


## hls4ml 

In [ ]:

# Based on the tutorials provided by hls4ml
# https://github.com/fastmachinelearning/hls4ml

hls4ml.model.optimizer.OutputRoundingSaturationMode.layers = ['Activation']
hls4ml.model.optimizer.OutputRoundingSaturationMode.rounding_mode = 'AP_RND'
hls4ml.model.optimizer.OutputRoundingSaturationMode.saturation_mode = 'AP_SAT'

hls_config = hls4ml.utils.config_from_keras_model(model, granularity='name')
hls_config['Model']['Precision'] = 'ap_fixed<8,4>'

for Layer in hls_config['LayerName'].keys():
    hls_config['LayerName'][Layer]['Strategy'] = 'Latency'
    hls_config['LayerName'][Layer]['ReuseFactor'] = 1

#hls_config['LayerName']['output_softmax']['Strategy'] = 'Stable'

cfg = hls4ml.converters.create_config(backend='Vivado')
cfg['IOType']     = 'io_stream' # Must set this if using CNNs!
cfg['HLSConfig']  = hls_config
cfg['KerasModel'] = model
cfg['OutputDir']  = 'model_IP/'
cfg['XilinxPart'] = 'xczu9eg-ffvb1156-2-e'  # PYNQ-Z1 or Zedboard: xc7z020-clg484-1
  
hls_model = hls4ml.converters.keras_to_hls(cfg)
hls_model.compile()

hls_model.build(csim=False, export=True)